# Step 1.1: Extracting Basic Features



Previously we have cleaned the data and exported a few clean datasets (in the **data/step1** folder).

- **biz.csv**: Businesses of 8 selected states, with neighborhood information derived from Zillow neighborhoods, and census tract information derived from geographical boundaries of census tracts. Some businesses may not be associated with any neighborhood name.
- **neighborhoods/neighborhoods.shp**: Shape file of all Zillow neighborhoods in the 8 states that have at least one business.
- **census-tracts/census-tracts.shp**: Shape file of all census tracts in the 8 states that have at least one business.

In this notebook, we aggregate a few neighborhood/census tract level measurements based on the Yelp data. Then we will apply clustering algorithms to these measurements later, to separate neighborhoods based on their business dynamics.

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.io.json import json_normalize

sns.set(
    style="white",
    color_codes=True,
    rc={
        'axes.linewidth': 0.5,
        'lines.linewidth': 2,
        'axes.labelsize': 14,
        'axes.titlesize': 14
    }
)

pd.set_option('display.max_rows', 4)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Basic features

In [21]:
# Load data
biz = pd.read_csv('../data/step1/biz.csv', dtype={
    'postal_code': str,
    'CT_ID': str
})
# drop those without Nhood information
# biz = biz[biz['Nhood'].notnull()]

In [22]:
biz.describe()

,is_open,review_count,stars,Canadian (New),RestaurantsPriceRange2,wday_hrs,wend_hrs,CT_LAND,CT_WATER,CT_BIZ_COUNT,Nhood_area,Nhood_biz_count,n_biz_in_2km,longitude,latitude
count,115522.000000,115522.000000,115522.000000,0.0,64888.000000,115522.000000,115522.000000,1.155220e+05,1.155220e+05,115522.000000,7.131500e+04,71315.000000,115522.000000,115522.000000,115522.000000
mean,0.848869,34.195876,3.681649,NaN,1.758399,40.837805,12.190735,1.094599e+07,1.784289e+05,170.122003,6.087613e+07,1656.134011,479.474074,-103.123968,36.094266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75%,1.000000,26.000000,4.500000,NaN,2.000000,56.000000,20.000000,5.591792e+06,1.298400e+04,171.000000,9.457853e+07,2882.000000,627.000000,-81.805939,36.217370
max,1.000000,6979.000000,5.000000,NaN,4.000000,120.000000,48.000000,2.147484e+09,2.904698e+08,1767.000000,2.707229e+08,5454.000000,3151.000000,-75.062989,46.504003


In [23]:
display(biz.columns)

Index(['address', 'business_id', 'city', 'is_open', 'name', 'neighborhood',
       'postal_code', 'review_count', 'stars', 'state', 'Active Life',
       'American (New)', 'American (Traditional)', 'Arts & Entertainment',
       'Asian Fusion', 'Automotive', 'Bakeries', 'Barbeque', 'Beauty & Spas',
       'Breakfast & Brunch', 'Buffets', 'Burgers', 'Cafes', 'Canadian (New)',
       'Chicken Wings', 'Chinese', 'Coffee & Tea', 'Convenience Stores',
       'Delis', 'Desserts', 'Diners', 'Event Planning & Services', 'Fast Food',
       'Food', 'French', 'Greek', 'Grocery', 'Health & Medical',
       'Home Services', 'Hotels & Travel', 'Ice Cream & Frozen Yogurt',
       'Indian', 'Italian', 'Japanese', 'Juice Bars & Smoothies', 'Korean',
       'Local Services', 'Mediterranean', 'Mexican', 'Middle Eastern',
       'Nightlife', 'Pets', 'Pizza', 'Professional Services', 'Restaurants',
       'Salad', 'Sandwiches', 'Seafood', 'Shopping', 'Specialty Food',
       'Steakhouses', 'Sushi Bars', '

In [24]:
display(biz.postal_code)

0         44143
1         28215
          ...  
115520    85281
115521    89014
Name: postal_code, Length: 115522, dtype: object

### Alcohol

Possible values for `Alcohol`

In [25]:
biz.Alcohol.unique()

array([nan, 'none', 'full_bar', 'beer_and_wine'], dtype=object)

In [26]:
# biz.loc[biz['Alcohol'] == 'none', 'Alcohol'] = np.nan
# biz.Alcohol.unique()

### Price range

In [27]:
biz.RestaurantsPriceRange2.values[:10]

array([  2.,   2.,  nan,   1.,   2.,  nan,  nan,   2.,  nan,  nan])

### Restaurant Attributes

There are a few restaurant only attributes.

- RestaurantsTakeOut
- RestaurantsGoodForGroups
- RestaurantsDelivery
- RestaurantsReservations
- RestaurantsAttire
- RestaurantsTableService
- RestaurantsCounterService

We would want to know how many restaurants have those attributes set.

In [28]:
def check_notnull_ratio(biz, attr1, attr2):
    return (biz[biz[attr1].notnull()][attr2].notnull()).mean()

rest_attrs = list(filter(lambda x: x.startswith('Restaurants'), biz.columns))
for attr in rest_attrs + ['OutdoorSeating', 'Alcohol']:
    print('{: <25} {}'.format(attr, check_notnull_ratio(biz, 'Restaurants', attr)))

Restaurants               1.0
RestaurantsPriceRange2    0.9395593745962011
RestaurantsGoodForGroups  0.919369427574622
RestaurantsDelivery       0.8995671275358573
RestaurantsTakeOut        0.9249580049101951
RestaurantsReservations   0.9021191368393849
RestaurantsAttire         0.9038635482620494
OutdoorSeating            0.8863225222897015
Alcohol                   0.8305336606796744


In [29]:
biz.RestaurantsAttire.unique()

array([nan, 'casual', 'dressy', 'formal'], dtype=object)

In [30]:
is_formal = biz['RestaurantsAttire'].isin(['dressy', 'formal'])
display(is_formal.mean())
is_formal[biz['RestaurantsAttire'].isnull()] = np.nan
display(is_formal.mean())

0.0060075137203303266

0.0247936836840413

In [31]:
# Must dress formal
dress_formal = biz['RestaurantsAttire'].isin(['dressy', 'formal'])
dress_formal[biz['RestaurantsAttire'].isnull()] = np.nan
biz['DressFormal'] = dress_formal

In [32]:
biz.OutdoorSeating.unique()

array([nan, False, True], dtype=object)

In [33]:
biz.RestaurantsTakeOut.unique()

array([nan, True, False], dtype=object)

## Build the feature matrix

After examing the extreme/unique values of the features, now aggregate the measurementns to neighborhoods and census tracts.

In [35]:
notnull_ratio = lambda x: x.notnull().mean()
def isval_ratio(y):
    def func(x):
        if x.notnull().any():
            return (x == y).mean() 
        return np.nan
    return func
def notval_ratio(y):
    def func(x):
        notnull = x.notnull()
        # we always exlude NA values for calculation
        if notnull.any():
            # not null and not equal
            return (notnull & (x != y)).mean() 
        return np.nan
    return func

def basic_agg(biz, by='Nhood'):
    """Calculate basic aggregates stats of the neighborhoods"""
    
    area_col = 'Nhood_area' if by == 'Nhood' else 'CT_LAND'
    nna_ratio = [('ratio', notnull_ratio)]
    avg = [('avg', 'mean')]
    
    stats = biz.groupby(by).agg({
        'business_id': ['count'],
        'n_biz_in_2km': avg,
        
        'is_open': avg,
        'stars': avg,
        'review_count': avg,
        'wday_hrs': avg,
        'wend_hrs': avg,
        'RestaurantsPriceRange2': avg,  # by default will skip NA
        
        'open_till_late': [('ratio', 'mean')], # avergage of (TRUE, FALSE) vals is ratio
        'Restaurants': nna_ratio,
        'Food': nna_ratio,
        'Coffee & Tea': nna_ratio,
        'Shopping': nna_ratio,
        'Nightlife': nna_ratio,
        'Home Services': nna_ratio,
        'Local Services': nna_ratio,
        'Beauty & Spas': nna_ratio,
        'Health & Medical': nna_ratio,
        'Automotive': nna_ratio,
        'Arts & Entertainment': nna_ratio,
        'Hotels & Travel': nna_ratio,
        
        # attributes
        'Alcohol': [('ratio', notval_ratio('none'))],
        'DressFormal': [('ratio', isval_ratio(1))],
        'RestaurantsTakeOut': [('ratio', isval_ratio(True))],
        # m^2 to km^2
        # area_col: [('km', lambda x: x.values[0] / 10e6)],
    }).sort_values(('n_biz_in_2km', 'avg'), ascending=False)
    
    stats.columns = stats.columns.map(lambda x: '_'.join(x).strip('_'))
    stats = pd.DataFrame(stats.to_records())
    return stats
    
nbh_stats_min_20 = basic_agg(biz[biz['Nhood_biz_count'] >= 20])
ct_stats_min_20 = basic_agg(biz[biz['CT_BIZ_COUNT'] >= 20], by='CT_ID')
display(nbh_stats_min_20)
display(ct_stats_min_20)

,Nhood,business_id_count,n_biz_in_2km_avg,is_open_avg,stars_avg,review_count_avg,wday_hrs_avg,wend_hrs_avg,RestaurantsPriceRange2_avg,open_till_late_ratio,...,Home Services_ratio,Local Services_ratio,Beauty & Spas_ratio,Health & Medical_ratio,Automotive_ratio,Arts & Entertainment_ratio,Hotels & Travel_ratio,Alcohol_ratio,DressFormal_ratio,RestaurantsTakeOut_ratio
0,"The Strip, Las Vegas, NV",2643,2522.919031,0.792660,3.570942,177.988271,41.209768,16.522783,2.342282,0.364359,...,0.006054,0.013243,0.072266,0.009837,0.004540,0.179342,0.082482,0.332577,0.04843,0.271283
1,"Meadows Village, Las Vegas, NV",426,1546.462441,0.786385,3.937793,57.819249,44.392567,14.867332,1.897119,0.215962,...,0.105634,0.105634,0.079812,0.023474,0.098592,0.119718,0.086854,0.136150,0.00939,0.157277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,"Foothills, Henderson, NV",39,29.487179,0.923077,3.974359,12.333333,43.352564,13.179487,1.666667,0.205128,...,0.282051,0.102564,0.102564,0.025641,0.128205,0.025641,0.025641,0.025641,0.00000,0.102564
314,"West Henderson, Henderson, NV",50,26.160000,0.920000,4.030000,103.240000,36.785000,13.750000,1.954545,0.260000,...,0.140000,0.120000,0.060000,0.040000,0.020000,0.100000,0.160000,0.260000,0.00000,0.220000


,CT_ID,business_id_count,n_biz_in_2km_avg,is_open_avg,stars_avg,review_count_avg,wday_hrs_avg,wend_hrs_avg,RestaurantsPriceRange2_avg,open_till_late_ratio,...,Home Services_ratio,Local Services_ratio,Beauty & Spas_ratio,Health & Medical_ratio,Automotive_ratio,Arts & Entertainment_ratio,Hotels & Travel_ratio,Alcohol_ratio,DressFormal_ratio,RestaurantsTakeOut_ratio
0,32003006700,1767,2742.401245,0.802490,3.594793,203.042445,43.269053,17.418487,2.371550,0.353707,...,0.007357,0.011885,0.063384,0.007357,0.003396,0.175439,0.074137,0.353707,0.052632,0.292586
1,32003002302,602,2404.418605,0.790698,3.665282,101.425249,37.732558,14.210548,2.299550,0.390365,...,0.009967,0.026578,0.109635,0.024917,0.011628,0.137874,0.093023,0.200997,0.031561,0.196013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,32003007500,60,2.766667,0.933333,4.075000,28.983333,36.050000,13.783333,1.923077,0.166667,...,0.066667,0.033333,0.000000,0.016667,0.000000,0.066667,0.183333,0.116667,0.000000,0.116667
1453,04013010102,29,1.758621,0.965517,4.224138,20.620690,20.086207,6.551724,2.000000,0.068966,...,0.137931,0.000000,0.034483,0.000000,0.034483,0.000000,0.172414,0.068966,0.000000,0.103448


## Export the feature matrix as a CSV file

These exported feature matrices will later be used in clustering.

In [36]:
nbh_stats_min_20.to_csv('../data/step2/nbh_stats_min_20.csv', index=False)
ct_stats_min_20.to_csv('../data/step2/ct_stats_min_20.csv', index=False)

In [37]:
nbh_stats_min_10 = basic_agg(biz[biz['Nhood_biz_count'] >= 10])
nbh_stats_min_10.to_csv('../data/step2/nbh_stats_min_10.csv', index=False)

ct_stats_min_10 = basic_agg(biz[biz['CT_BIZ_COUNT'] >= 10], by='CT_ID')
ct_stats_min_10.to_csv('../data/step2/ct_stats_min_10.csv', index=False)